# Laboratorio MongoDB

Run this code in Google Colab with CTRL + F9 and explain in your own words in a PDF 1/2 pages what the code is doing.



In [1]:
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | sudo tee /usr/share/keyrings/mongodb-server-6.0.asc
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.asc ] https://repo.mongodb.org/apt/ubuntu $(lsb_release -cs)/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!sudo apt update
!apt install -y mongodb-org mongodb-mongosh

-----BEGIN PGP PUBLIC KEY BLOCK-----
Version: GnuPG v1.4.11 (GNU/Linux)

mQINBGIWTroBEADgSBs1z1MC5Hog5yd2wYHskzPE0SOl9LGB35Xhw1894hrKsswp
AS7JnViltXE71iJMoAqepJBvfmZLOyQO0rXcLlHXExK/IctnosRqGQeyLxNZKS0h
e1xQYQrPCWRaHqseYLuJ5wME49aFQ2YS7caFowBvKjsT5AoT7B0uXDp6nHZDUQG2
MBZJqUKziVYYt7PARv81llDNKqPvLDSc2McL/2aa4mNR/pM5r8iQjACbSnj37ERm
zca2gJ0GzCeZSqfmjoF7I6Ez1Nc/2ge1+fZA24pDFg+7W25du3JIqbnpJQAK5TAz
7tVzvEKU8WT9aQW3G1e5ox3YtlRPTSrTxN9dzLh123NGCd0J9a4moFkZIr8HmySd
jkdz4V1pKv9aTOhLjQpF/bhRaUuNuGK7TV7ZzY+PCVE51fmJx2EX4Ck5c6sW03rJ
59KbrxeTq02AcIBTFUY0Mfh7nxvYvwvLI0OKBOqFGXi4hFXpV4uo0rDLe+tGLFDD
+HsajFUUyAlMETE80PXOuTs44TZiW+SGCTyP2Sm8TBIiacSqsGNsryjgEDaIG6c1
FB++njqTfGlyZujamYbF3s3wBK8nDBVRympJcsHjLqUhvbh1Bq4hyF2pxio93SgA
mPEm6kl0KBCqpJNZpAFSVHK8penQtQUa0jFQetYPDUFfgTsg7qdZDQNcUwARAQAB
tDdNb25nb0RCIDYuMCBSZWxlYXNlIFNpZ25pbmcgS2V5IDxwYWNrYWdpbmdAbW9u
Z29kYi5jb20+iQI+BBMBAgAoBQJiFk66AhsDBQkJZgGABgsJCAcDAgYVCAIJCgsE
FgIDAQIeAQIXgAAKCRBqJrGuZMPDiADhEACex1qu1HbVIeBwZO4GYYEc8OpswguI
LvTL1ufWMVbpSFkm0

In [2]:
import subprocess
import signal

# Define a custom exception for timeout
class TimeoutException(Exception):
    pass

# Define a handler that will be called when the alarm is triggered
def timeout_handler(signum, frame):
    raise TimeoutException("Code execution timed out!")

# Function to run a command with a timeout
def run_command_with_timeout(command, timeout):
    try:
        # Set the signal handler and an alarm
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        # Execute the shell command
        process = subprocess.run(command, shell=True, check=True)

        # Cancel the alarm if the process completes in time
        signal.alarm(0)
        return process.returncode
    except TimeoutException:
        print("Command timed out!")
        return None
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e}")
        return None
    finally:
        # Disable the alarm
        signal.alarm(0)

# Commands to run
commands = [
    "mkdir -p /data/db",
    "mongod --dbpath /data/db --bind_ip 0.0.0.0 --fork --logpath /data/db/mongodb.log",
    "mongosh"
]

# Run each command with a timeout
for cmd in commands:
    print(f"Running command: {cmd}")
    run_command_with_timeout(cmd, timeout=10)


Running command: mkdir -p /data/db
Running command: mongod --dbpath /data/db --bind_ip 0.0.0.0 --fork --logpath /data/db/mongodb.log
Command failed with error: Command 'mongod --dbpath /data/db --bind_ip 0.0.0.0 --fork --logpath /data/db/mongodb.log' returned non-zero exit status 48.
Running command: mongosh


In [3]:
!pip install pymongo

In [4]:
import pymongo
import pandas as pd
from pymongo import MongoClient

# Initialize MongoDB client and create a local database and collection
client = MongoClient("mongodb://localhost:27017/")
db = client["sample_db"]
collection = db["products"]
client.server_info()

# Clear existing data in the collection if it exists
collection.delete_many({})

# Sample data to insert into the MongoDB collection
data = [
    {"product_id": 1, "name": "Laptop", "category": "Electronics", "price": 1200, "quantity": 10},
    {"product_id": 2, "name": "Smartphone", "category": "Electronics", "price": 800, "quantity": 25},
    {"product_id": 3, "name": "Tablet", "category": "Electronics", "price": 600, "quantity": 15},
    {"product_id": 4, "name": "Headphones", "category": "Accessories", "price": 150, "quantity": 50},
    {"product_id": 5, "name": "Smartwatch", "category": "Accessories", "price": 300, "quantity": 30},
    {"product_id": 6, "name": "Camera", "category": "Photography", "price": 1000, "quantity": 8},
    {"product_id": 7, "name": "Tripod", "category": "Photography", "price": 100, "quantity": 20},
    {"product_id": 8, "name": "Monitor", "category": "Electronics", "price": 400, "quantity": 12},
    {"product_id": 9, "name": "Keyboard", "category": "Accessories", "price": 50, "quantity": 100},
    {"product_id": 10, "name": "Mouse", "category": "Accessories", "price": 40, "quantity": 75}
]

# Insert sample data into the collection
collection.insert_many(data)
print("Sample data inserted successfully!")


Sample data inserted successfully!


## Running MongoDB Queries
### 1. Finding all products in the Electronics category:

In [5]:
# Function to convert MongoDB query results to DataFrame for display
def to_dataframe(cursor):
    return pd.DataFrame(list(cursor))

# Simple Query - Find all products in Electronics category
electronics_products = collection.find({"category": "Electronics"})
electronics_df = to_dataframe(electronics_products)
electronics_df


,_id,product_id,name,category,price,quantity
0,6734ee4cf7637909a0c02fa5,1,Laptop,Electronics,1200,10
1,6734ee4cf7637909a0c02fa6,2,Smartphone,Electronics,800,25
2,6734ee4cf7637909a0c02fa7,3,Tablet,Electronics,600,15
3,6734ee4cf7637909a0c02fac,8,Monitor,Electronics,400,12


### 2. Aggregation Pipeline - Match, Sort, Group, and Limit

In [6]:

# Aggregation Pipeline Example
pipeline = [
    {"$match": {"category": "Accessories"}},
    {"$sort": {"price": -1}},
    {"$group": {"_id": "$category", "total_quantity": {"$sum": "$quantity"}, "average_price": {"$avg": "$price"}}},
    {"$limit": 2}
]

aggregation_result = collection.aggregate(pipeline)
aggregation_df = to_dataframe(aggregation_result)
aggregation_df


,_id,total_quantity,average_price
0,Accessories,255,135.0


## 3. Updating Documents
### Using `$set` to Update a Document

In [7]:

# Update the price of the product with product_id 1 (Laptop)
update_result = collection.update_one({"product_id": 1}, {"$set": {"price": 1100}})
print(f"Matched {update_result.matched_count} document(s) and modified {update_result.modified_count} document(s).")

# Fetch the updated document
updated_product = collection.find_one({"product_id": 1})
updated_product


Matched 1 document(s) and modified 1 document(s).


{'_id': ObjectId('6734ee4cf7637909a0c02fa5'),
 'product_id': 1,
 'name': 'Laptop',
 'category': 'Electronics',
 'price': 1100,
 'quantity': 10}

### 4. Deleting Documents
#### Deleting Documents with a Specific Condition

In [8]:

# Delete products in the 'Photography' category
delete_result = collection.delete_many({"category": "Photography"})
print(f"Deleted {delete_result.deleted_count} document(s).")

# Verify remaining documents
remaining_products = collection.find()
to_dataframe(remaining_products)


Deleted 2 document(s).


,_id,product_id,name,category,price,quantity
0,6734ee4cf7637909a0c02fa5,1,Laptop,Electronics,1100,10
1,6734ee4cf7637909a0c02fa6,2,Smartphone,Electronics,800,25
2,6734ee4cf7637909a0c02fa7,3,Tablet,Electronics,600,15
3,6734ee4cf7637909a0c02fa8,4,Headphones,Accessories,150,50
4,6734ee4cf7637909a0c02fa9,5,Smartwatch,Accessories,300,30
5,6734ee4cf7637909a0c02fac,8,Monitor,Electronics,400,12
6,6734ee4cf7637909a0c02fad,9,Keyboard,Accessories,50,100
7,6734ee4cf7637909a0c02fae,10,Mouse,Accessories,40,75


## 5. Creating Indexes
### Creating an Index on the `name` Field

In [9]:

# Create an index on the 'name' field to optimize search queries
index_result = collection.create_index("name")
print(f"Index created: {index_result}")

# List all indexes
indexes = collection.index_information()
indexes


Index created: name_1


{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_1': {'v': 2, 'key': [('name', 1)]}}

## 6. Using Text Search
### Text Search on Product Names

In [10]:

# Add a text index on the 'name' field for text search
collection.create_index([("name", "text")])

# Use text search to find products containing the word 'Smart'
text_search_result = collection.find({"$text": {"$search": "Smart"}})
to_dataframe(text_search_result)


""


## 7. Using `$lookup` for Joins
### Performing a Join between Two Collections

In [11]:

# Create another collection for orders
orders_collection = db["orders"]
orders_collection.delete_many({})  # Clear existing data

# Sample order data
orders_data = [
    {"order_id": 1, "product_id": 1, "quantity": 2},
    {"order_id": 2, "product_id": 2, "quantity": 1},
    {"order_id": 3, "product_id": 5, "quantity": 4}
]
orders_collection.insert_many(orders_data)

# Perform a $lookup to join products with orders
pipeline = [
    {
        "$lookup": {
            "from": "orders",
            "localField": "product_id",
            "foreignField": "product_id",
            "as": "order_info"
        }
    },
    {"$match": {"order_info": {"$ne": []}}}
]

join_result = collection.aggregate(pipeline)
to_dataframe(join_result)


,_id,product_id,name,category,price,quantity,order_info
0,6734ee4cf7637909a0c02fa5,1,Laptop,Electronics,1100,10,"[{'_id': 6734ee4cf7637909a0c02faf, 'order_id':..."
1,6734ee4cf7637909a0c02fa6,2,Smartphone,Electronics,800,25,"[{'_id': 6734ee4cf7637909a0c02fb0, 'order_id':..."
2,6734ee4cf7637909a0c02fa9,5,Smartwatch,Accessories,300,30,"[{'_id': 6734ee4cf7637909a0c02fb1, 'order_id':..."


## 8. Using `$project` to Control Output Fields
### Including Only Specific Fields in the Result

In [12]:

# Use $project to include only 'name' and 'price' fields in the output
project_result = collection.aggregate([
    {"$project": {"_id": 0, "name": 1, "price": 1}}
])
to_dataframe(project_result)


,name,price
0,Laptop,1100
1,Smartphone,800
2,Tablet,600
3,Headphones,150
4,Smartwatch,300
5,Monitor,400
6,Keyboard,50
7,Mouse,40


## 9. Using `$unwind` to Flatten Arrays
### Flattening the `order_info` Array

In [13]:

# Use $unwind to flatten the 'order_info' array
unwind_result = collection.aggregate([
    {
        "$lookup": {
            "from": "orders",
            "localField": "product_id",
            "foreignField": "product_id",
            "as": "order_info"
        }
    },
    {"$unwind": "$order_info"}
])
to_dataframe(unwind_result)


,_id,product_id,name,category,price,quantity,order_info
0,6734ee4cf7637909a0c02fa5,1,Laptop,Electronics,1100,10,"{'_id': 6734ee4cf7637909a0c02faf, 'order_id': ..."
1,6734ee4cf7637909a0c02fa6,2,Smartphone,Electronics,800,25,"{'_id': 6734ee4cf7637909a0c02fb0, 'order_id': ..."
2,6734ee4cf7637909a0c02fa9,5,Smartwatch,Accessories,300,30,"{'_id': 6734ee4cf7637909a0c02fb1, 'order_id': ..."


## 10. Using `$regex` for Pattern Matching
### Finding Products with a Name Matching a Pattern

In [14]:

# Find products where the name starts with 'S'
regex_result = collection.find({"name": {"$regex": "^S"}})
to_dataframe(regex_result)


,_id,product_id,name,category,price,quantity
0,6734ee4cf7637909a0c02fa6,2,Smartphone,Electronics,800,25
1,6734ee4cf7637909a0c02fa9,5,Smartwatch,Accessories,300,30
